# univ3-da-ezkl

Here's an example leveraging EZKL whereby the inputs to the model are read and attested to from an on-chain source.

In this setup:
- the inputs and outputs are publicly known to the prover and verifier
- the on chain inputs will be fetched and then fed directly into the circuit
- the quantization of the on-chain inputs happens within the evm and is replicated at proving time 


First we import the necessary dependencies and set up logging to be as informative as possible. 

In [11]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


from torch import nn
import ezkl
import os
import json
import logging

# uncomment for more descriptive logging 
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.DEBUG)


Now we define our model. It is a very simple PyTorch model that has just one layer, an average pooling 2D layer. 

In [12]:
import torch
# Defines the model

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer = nn.AvgPool2d(2, 1, (1, 1))

    def forward(self, x):
        return self.layer(x)[0]


circuit = MyModel()

# this is where you'd train your model

We omit training for purposes of this demonstration. We've marked where training would happen in the cell above. 
Now we export the model to onnx and create a corresponding (randomly generated) input. This input data will eventually be stored on chain and read from according to the call_data field in the graph input.

You can replace the random `x` with real data if you so wish. 

In [13]:
x = 0.1*torch.rand(1,*[3, 2, 2], requires_grad=True)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      "network.onnx",            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump(data, open("input.json", 'w' ))



We now define a function that will create a new anvil instance which we will deploy our test contract too. This contract will contain in its storage the data that we will read from and attest to. In production you would not need to set up a local anvil instance. Instead you would replace RPC_URL with the actual RPC endpoint of the chain you are deploying your verifiers too, reading from the data on said chain.

In [14]:
import subprocess
import time
import threading

# make sure anvil is running locally
# $ anvil -p 3030

RPC_URL = "http://localhost:3030"

# Save process globally
anvil_process = None

def start_anvil():
    global anvil_process
    if anvil_process is None:
        anvil_process = subprocess.Popen(["anvil", "-p", "3030", "--fork-url", "https://arb1.arbitrum.io/rpc", "--code-size-limit=41943040"])
        if anvil_process.returncode is not None:
            raise Exception("failed to start anvil process")
        time.sleep(3)

def stop_anvil():
    global anvil_process
    if anvil_process is not None:
        anvil_process.terminate()
        anvil_process = None


We define our `PyRunArgs` objects which contains the visibility parameters for out model. 
- `input_visibility` defines the visibility of the model inputs
- `param_visibility` defines the visibility of the model weights and constants and parameters 
- `output_visibility` defines the visibility of the model outputs

Here we create the following setup:
- `input_visibility`: "public"
- `param_visibility`: "private"
- `output_visibility`: public


In [15]:
import ezkl

model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
data_path = os.path.join('input.json')

run_args = ezkl.PyRunArgs()
run_args.input_visibility = "public"
run_args.param_visibility = "private"
run_args.output_visibility = "public"
run_args.num_inner_cols = 1
run_args.variables = [("batch_size", 1)]

Now we generate a settings file. This file basically instantiates a bunch of parameters that determine their circuit shape, size etc... Because of the way we represent nonlinearities in the circuit (using Halo2's [lookup tables](https://zcash.github.io/halo2/design/proving-system/lookup.html)), it is often best to _calibrate_ this settings file as some data can fall out of range of these lookups.

You can pass a dataset for calibration that will be representative of real inputs you might find if and when you deploy the prover. Here we create a dummy calibration dataset for demonstration purposes. 

In [16]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path, py_run_args=run_args)
assert res == True

DEBUG tract_onnx.model 2024-11-08 06:02:15,077 model.rs:254 ONNX operator set version: 10
DEBUG ezkl.graph.model 2024-11-08 06:02:15,078 model.rs:661 set batch_size to 1
DEBUG tract_hir.infer.analyser 2024-11-08 06:02:15,078 analyser.rs:151   Refined 1/0>: ..,? -> batch_size,3,3,3,F32
DEBUG tract_hir.infer.analyser 2024-11-08 06:02:15,078 analyser.rs:151   Refined 2/0>: ..,? -> ,I64 0
DEBUG tract_hir.infer.analyser 2024-11-08 06:02:15,079 analyser.rs:151   Refined 3/0>: ..,? -> 3,3,3,F32
DEBUG tract_core.optim 2024-11-08 06:02:15,079 mod.rs:174 applying patch #0: declutter/0 >> declutter #1 "/layer/AveragePool" SumPool
DEBUG tract_core.optim.change_axes 2024-11-08 06:02:15,079 change_axes.rs:83   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2024-11-08 06:02:15,079 change_axes.rs:95     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2024-11-08 06:02:15,079 change_axes.rs:83

In [17]:
# generate a bunch of dummy calibration data
cal_data = {
    "input_data": [(0.1*torch.rand(2, *[3, 2, 2])).flatten().tolist()],
}

cal_path = os.path.join('val_data.json')
# save as json file
with open(cal_path, "w") as f:
    json.dump(cal_data, f)

res = await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")

DEBUG tract_onnx.model 2024-11-08 06:02:15,096 model.rs:254 ONNX operator set version: 10
DEBUG ezkl.graph.model 2024-11-08 06:02:15,096 model.rs:661 set batch_size to 1
DEBUG tract_hir.infer.analyser 2024-11-08 06:02:15,097 analyser.rs:151   Refined 1/0>: ..,? -> batch_size,3,3,3,F32
DEBUG tract_hir.infer.analyser 2024-11-08 06:02:15,097 analyser.rs:151   Refined 2/0>: ..,? -> ,I64 0
DEBUG tract_hir.infer.analyser 2024-11-08 06:02:15,098 analyser.rs:151   Refined 3/0>: ..,? -> 3,3,3,F32
DEBUG tract_core.optim 2024-11-08 06:02:15,098 mod.rs:174 applying patch #0: declutter/0 >> declutter #1 "/layer/AveragePool" SumPool
DEBUG tract_core.optim.change_axes 2024-11-08 06:02:15,099 change_axes.rs:83   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2024-11-08 06:02:15,099 change_axes.rs:95     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2024-11-08 06:02:15,100 change_axes.rs:83

In [18]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

DEBUG tract_onnx.model 2024-11-08 06:02:18,483 model.rs:254 ONNX operator set version: 10
DEBUG ezkl.graph.model 2024-11-08 06:02:18,484 model.rs:661 set batch_size to 1
DEBUG tract_hir.infer.analyser 2024-11-08 06:02:18,484 analyser.rs:151   Refined 1/0>: ..,? -> batch_size,3,3,3,F32
DEBUG tract_hir.infer.analyser 2024-11-08 06:02:18,484 analyser.rs:151   Refined 2/0>: ..,? -> ,I64 0
DEBUG tract_hir.infer.analyser 2024-11-08 06:02:18,484 analyser.rs:151   Refined 3/0>: ..,? -> 3,3,3,F32
DEBUG tract_core.optim 2024-11-08 06:02:18,485 mod.rs:174 applying patch #0: declutter/0 >> declutter #1 "/layer/AveragePool" SumPool
DEBUG tract_core.optim.change_axes 2024-11-08 06:02:18,485 change_axes.rs:83   Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG tract_core.optim.change_axes 2024-11-08 06:02:18,485 change_axes.rs:95     Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2024-11-08 06:02:18,485 change_axes.rs:83

The graph input for on chain data sources is formatted completely differently compared to file based data sources.

- For file data sources, the raw floating point values that eventually get quantized, converted into field elements and stored in `witness.json` to be consumed by the circuit are stored. The output data contains the expected floating point values returned as outputs from running your vanilla pytorch model on the given inputs.
- For on chain data sources, the input_data field contains all the data necessary to read and format the on chain data into something digestable by EZKL (aka field elements :-D). 
Here is what the schema for an on-chain data source graph input file should look like:
    
```json
{
  "input_data": {
    "rpc": "http://localhost:3030", // The rpc endpoint of the chain you are deploying your verifier to
    "calls": [
      {
        "call_data": [
          [
            "71e5ee5f0000000000000000000000000000000000000000000000000000000000000000", // The abi encoded call data to a view function that returns a single on-chain data point (we only support uint256 returns for now)
            7 // The number of decimal places of the large uint256 value. This is our way of representing large wei values as floating points on chain, since the evm only natively supports integer values.
          ],
          [
            "71e5ee5f0000000000000000000000000000000000000000000000000000000000000001",
            5
          ],
          [
            "71e5ee5f0000000000000000000000000000000000000000000000000000000000000002",
            5
          ]
        ],
        "address": "5fbdb2315678afecb367f032d93f642f64180aa3" // The address of the contract that we are calling to get the data. 
      }
    ]
  }
}

In [19]:
from web3 import Web3, HTTPProvider
from solcx import compile_standard
from decimal import Decimal
import json
import os
import torch

# This function counts the decimal places of a floating point number
def count_decimal_places(num):
    num_str = str(num)
    if '.' in num_str:
        return len(num_str) - 1 - num_str.index('.')
    else:
        return 0

# setup web3 instance
w3 = Web3(HTTPProvider(RPC_URL)) 

def on_chain_data(tensor):
    # Step 0: Convert the tensor to a flat list
    data = tensor.view(-1).tolist()

    # Step 1: Prepare the calldata
    secondsAgo = [len(data) - 1 - i for i in range(len(data))]

    # Step 2: Prepare and compile the contract UniTickAttestor contract
    contract_source_code = '''
    // SPDX-License-Identifier: MIT
    pragma solidity ^0.8.20;

    /// @title Pool state that is not stored
    /// @notice Contains view functions to provide information about the pool that is computed rather than stored on the
    /// blockchain. The functions here may have variable gas costs.
    interface IUniswapV3PoolDerivedState {
        /// @notice Returns the cumulative tick and liquidity as of each timestamp `secondsAgo` from the current block timestamp
        /// @dev To get a time weighted average tick or liquidity-in-range, you must call this with two values, one representing
        /// the beginning of the period and another for the end of the period. E.g., to get the last hour time-weighted average tick,
        /// you must call it with secondsAgos = [3600, 0].
        /// log base sqrt(1.0001) of token1 / token0. The TickMath library can be used to go from a tick value to a ratio.
        /// @dev The time weighted average tick represents the geometric time weighted average price of the pool, in
        /// @param secondsAgos From how long ago each cumulative tick and liquidity value should be returned
        /// @return tickCumulatives Cumulative tick values as of each `secondsAgos` from the current block timestamp
        /// @return secondsPerLiquidityCumulativeX128s Cumulative seconds per liquidity-in-range value as of each `secondsAgos` from the current block
        /// timestamp
        function observe(
            uint32[] calldata secondsAgos
        )
            external
            view
            returns (
                int56[] memory tickCumulatives,
                uint160[] memory secondsPerLiquidityCumulativeX128s
            );
    }

    /// @title Uniswap Wrapper around `pool.observe` that stores the parameters for fetching and then attesting to historical data
    /// @notice Provides functions to integrate with V3 pool oracle
    contract UniTickAttestor {
        /**
        * @notice Calculates time-weighted means of tick and liquidity for a given Uniswap V3 pool
        * @param pool Address of the pool that we want to observe
        * @param secondsAgo Number of seconds in the past from which to calculate the time-weighted means
        * @return tickCumulatives The cumulative tick values as of each `secondsAgo` from the current block timestamp
        */
        function consult(
            IUniswapV3PoolDerivedState pool,
            uint32[] memory secondsAgo
        ) public view returns (int256[] memory tickCumulatives) {
            (int56[] memory _tickCumulatives, ) = pool.observe(secondsAgo);
            tickCumulatives = new int256[](_tickCumulatives.length);
            for (uint256 i = 0; i < _tickCumulatives.length; i++) {
                tickCumulatives[i] = int256(_tickCumulatives[i]);
            }
        }
    }
    '''

    compiled_sol = compile_standard({
        "language": "Solidity",
        "sources": {"UniTickAttestor.sol": {"content": contract_source_code}},
        "settings": {"outputSelection": {"*": {"*": ["metadata", "evm.bytecode", "abi"]}}}
    })

    # Get bytecode
    bytecode = compiled_sol['contracts']['UniTickAttestor.sol']['UniTickAttestor']['evm']['bytecode']['object']

    # Get ABI
    # In production if you are reading from really large contracts you can just use
    # a stripped down version of the ABI of the contract you are calling, containing only the view functions you will fetch data from.
    abi = json.loads(compiled_sol['contracts']['UniTickAttestor.sol']['UniTickAttestor']['metadata'])['output']['abi']

    # Step 3: Deploy the contract
    UniTickAttestor = w3.eth.contract(abi=abi, bytecode=bytecode)
    tx_hash = UniTickAttestor.constructor().transact()
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    # If you are deploying to production you can skip the 3 lines of code above and just instantiate the contract like this,
    # passing the address and abi of the contract you are fetching data from.
    contract = w3.eth.contract(address=tx_receipt['contractAddress'], abi=abi)

    # Step 4: Interact with the contract
    call = contract.functions.consult(
        # Address of the UniV3 usdc-weth pool 0.005 fee
        "0xC6962004f452bE9203591991D15f6b388e09E8D0",
        secondsAgo
    ).build_transaction()
    result = contract.functions.consult(
        # Address of the UniV3 usdc-weth pool 0.005 fee
        "0xC6962004f452bE9203591991D15f6b388e09E8D0",
        secondsAgo
    ).call()
    print(f'result: {result}')
    calldata = call['data'][2:]


    # Prepare the calls_to_account object
    # If you were calling view functions across multiple contracts,
    # you would have multiple entries in the calls_to_account array,
    # one for each contract.
    call_to_account = {
        'call_data': calldata,
        'decimals': 0,
        'address': contract.address[2:], # remove the '0x' prefix
        'len': len(data),
    }

    print(f'call_to_account: {call_to_account}')

    return call_to_account

# Now let's start the Anvil process. You don't need to do this if you are deploying to a non-local chain.
start_anvil()

# Now let's call our function, passing in the same input tensor we used to export the model 2 cells above.
calls_to_account = on_chain_data(x)

data = dict(input_data =  {'rpc': RPC_URL, 'calls': calls_to_account })

# Serialize on-chain data into file:
json.dump(data, open("input.json", 'w'))

Error: Address already in use (os error 48)

Location:
    /Users/ethancemer/.cargo/git/checkouts/foundry-f7cca724e93059b0/62cdea8/crates/anvil/src/cmd.rs:283:33
DEBUG web3.manager.RequestManager 2024-11-08 06:02:21,545 manager.py:331 Making request. Method: eth_sendTransaction
DEBUG web3.manager.RequestManager 2024-11-08 06:02:21,545 manager.py:331 Making request. Method: eth_getBlockByNumber
DEBUG web3.providers.HTTPProvider 2024-11-08 06:02:21,546 rpc.py:166 Making request HTTP. URI: http://localhost:3030, Method: eth_getBlockByNumber
DEBUG web3._utils.http_session_manager.HTTPSessionManager 2024-11-08 06:02:21,546 http_session_manager.py:83 Session cached: http://localhost:3030, <requests.sessions.Session object at 0x328834210>
DEBUG urllib3.connectionpool 2024-11-08 06:02:21,548 connectionpool.py:243 Starting new HTTP connection (1): localhost:3030


eth_getBlockByNumber


DEBUG urllib3.connectionpool 2024-11-08 06:02:21,910 connectionpool.py:546 http://localhost:3030 "POST / HTTP/11" 200 1852
DEBUG web3.providers.HTTPProvider 2024-11-08 06:02:21,910 rpc.py:172 Getting response HTTP. URI: http://localhost:3030, Method: eth_getBlockByNumber, Response: {'jsonrpc': '2.0', 'id': 0, 'result': {'hash': '0x071a9d505b4c649bbc037b809c146bd2c0c2baf85ef411344fb4fc00ea12be97', 'parentHash': '0x682b8a499bc9cdc5bc91af84c266f6ec841391bf9e038b1878088c6c597483eb', 'sha3Uncles': '0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347', 'miner': '0xa4b000000000000000000073657175656e636572', 'stateRoot': '0x26fd0acf39984ef7f80722b561d5ba67534aae32d6c0264e9095ec0a91f10760', 'transactionsRoot': '0x8d391b6f1b75f1801c16be33f99f9d08e9495fb0a113348a25af56227bfe6835', 'receiptsRoot': '0x37cd4be5c0b3df930195268e9cf2c7cb2d764daef5c8b1e7d43832b783682886', 'logsBloom': '0x000080000020000000010000000000000000000004000000000000000000000000001000000000000000000000000000000000

eth_chainId
eth_chainId
eth_estimateGas


DEBUG urllib3.connectionpool 2024-11-08 06:02:22,650 connectionpool.py:546 http://localhost:3030 "POST / HTTP/11" 200 43
DEBUG web3.providers.HTTPProvider 2024-11-08 06:02:22,652 rpc.py:172 Getting response HTTP. URI: http://localhost:3030, Method: eth_estimateGas, Response: {'jsonrpc': '2.0', 'id': 3, 'result': '0x845a8'}
DEBUG web3.manager.RequestManager 2024-11-08 06:02:22,653 manager.py:331 Making request. Method: eth_chainId
DEBUG web3.providers.HTTPProvider 2024-11-08 06:02:22,653 rpc.py:166 Making request HTTP. URI: http://localhost:3030, Method: eth_chainId
DEBUG urllib3.connectionpool 2024-11-08 06:02:22,655 connectionpool.py:546 http://localhost:3030 "POST / HTTP/11" 200 42
DEBUG web3.providers.HTTPProvider 2024-11-08 06:02:22,656 rpc.py:172 Getting response HTTP. URI: http://localhost:3030, Method: eth_chainId, Response: {'jsonrpc': '2.0', 'id': 4, 'result': '0xa4b1'}
DEBUG web3.manager.RequestManager 2024-11-08 06:02:22,656 manager.py:331 Making request. Method: eth_getBloc

...

eth_getTransactionReceipt
eth_getTransactionReceipt

    Transaction: 0x371f06c0c1625619151e8706d8132754ae01e1e0ffaf7e514d016aae3c941bea
    Contract created: 0x90dd5250fd06b9e6e3d048caf7f26da609cb67cc
    Gas used: 542120

    Block Number: 272110988
    Block Hash: 0xc4868a5363efcbfc104b8ceff2a2a05bb2acd83b1477b780a3c5b4a2ac6fc52a
    Block Time: "Thu, 7 Nov 2024 22:02:19 +0000"



DEBUG web3.manager.RequestManager 2024-11-08 06:02:23,105 manager.py:331 Making request. Method: eth_getTransactionReceipt
DEBUG web3.providers.HTTPProvider 2024-11-08 06:02:23,106 rpc.py:166 Making request HTTP. URI: http://localhost:3030, Method: eth_getTransactionReceipt
DEBUG urllib3.connectionpool 2024-11-08 06:02:23,107 connectionpool.py:546 http://localhost:3030 "POST / HTTP/11" 200 1125
DEBUG web3.providers.HTTPProvider 2024-11-08 06:02:23,108 rpc.py:172 Getting response HTTP. URI: http://localhost:3030, Method: eth_getTransactionReceipt, Response: {'jsonrpc': '2.0', 'id': 12, 'result': {'type': '0x2', 'status': '0x1', 'cumulativeGasUsed': '0x845a8', 'logs': [], 'logsBloom': '0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

eth_getTransactionReceipt
eth_chainId
eth_estimateGas


DEBUG urllib3.connectionpool 2024-11-08 06:02:24,791 connectionpool.py:546 http://localhost:3030 "POST / HTTP/11" 200 44
DEBUG web3.providers.HTTPProvider 2024-11-08 06:02:24,792 rpc.py:172 Getting response HTTP. URI: http://localhost:3030, Method: eth_estimateGas, Response: {'jsonrpc': '2.0', 'id': 14, 'result': '0x13d39'}
DEBUG web3.manager.RequestManager 2024-11-08 06:02:24,792 manager.py:331 Making request. Method: eth_chainId
DEBUG web3.providers.HTTPProvider 2024-11-08 06:02:24,792 rpc.py:166 Making request HTTP. URI: http://localhost:3030, Method: eth_chainId
DEBUG urllib3.connectionpool 2024-11-08 06:02:24,794 connectionpool.py:546 http://localhost:3030 "POST / HTTP/11" 200 43
DEBUG web3.providers.HTTPProvider 2024-11-08 06:02:24,794 rpc.py:172 Getting response HTTP. URI: http://localhost:3030, Method: eth_chainId, Response: {'jsonrpc': '2.0', 'id': 15, 'result': '0xa4b1'}
DEBUG web3.manager.RequestManager 2024-11-08 06:02:24,794 manager.py:331 Making request. Method: eth_maxPr

eth_chainId
eth_maxPriorityFeePerGas
eth_getBlockByNumber
eth_chainId
eth_chainId
eth_call
eth_chainId
result: [-8877442833978, -8877443030596, -8877443227214, -8877443423832, -8877443620450, -8877443817068, -8877444013686, -8877444210304, -8877444406922, -8877444603540, -8877444800158, -8877444996776]
call_to_account: {'call_data': '1f3be514000000000000000000000000c6962004f452be9203591991d15f6b388e09e8d00000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000000b000000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000000900000000000000000000000000000000000000000000000000000000000000080000000000000000000000000000000000000000000000000000000000000007000000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000050000000000000000

As we use Halo2 with KZG-commitments we need an SRS string from (preferably) a multi-party trusted setup ceremony. For an overview of the procedures for such a ceremony check out [this page](https://blog.ethereum.org/2023/01/16/announcing-kzg-ceremony). The `get_srs` command retrieves a correctly sized SRS given the calibrated settings file from [here](https://github.com/han0110/halo2-kzg-srs). 

These SRS were generated with [this](https://github.com/privacy-scaling-explorations/perpetualpowersoftau) ceremony. 

In [20]:
res = await ezkl.get_srs( settings_path)

INFO ezkl.execute 2024-11-08 06:02:24,812 execute.rs:699 SRS already exists at that path
INFO ezkl.execute 2024-11-08 06:02:24,813 execute.rs:598 read 33028 bytes from file (vector of len = 33028)
INFO ezkl.execute 2024-11-08 06:02:24,814 execute.rs:605 file hash: 446092fd1d6030e5bb2f2a8368267d5ed0fbdb6a766f6c5e4a4841827ad3106f


We now need to generate the circuit witness. These are the model outputs (and any hashes) that are generated when feeding the previously generated `input.json` through the circuit / model. 

In [21]:
!export RUST_BACKTRACE=1

witness_path = "witness.json"

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)

DEBUG ezkl.graph 2024-11-08 06:02:25,102 mod.rs:944 input scales: [13]
DEBUG alloy_rpc_client.call 2024-11-08 06:02:25,103 call.rs:85 sending request method=eth_chainId id=0
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:25,103 reqwest_transport.rs:24 ReqwestTransport; url=http://localhost:3030/
DEBUG reqwest.connect 2024-11-08 06:02:25,104 connect.rs:497 starting new connection: http://localhost:3030/
DEBUG hyper_util.client.legacy.connect.dns 2024-11-08 06:02:25,104 dns.rs:122 resolving host="localhost"
DEBUG hyper_util.client.legacy.connect.http 2024-11-08 06:02:25,105 http.rs:643 connecting to 127.0.0.1:3030
DEBUG hyper_util.client.legacy.connect.http 2024-11-08 06:02:25,106 http.rs:646 connected to 127.0.0.1:3030
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:02:25,106 pool.rs:396 pooling idle connection for ("http", localhost:3030)
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:25,107 reqwest_transport.rs:36 received response from server status=20

eth_chainId
eth_call
eth_estimateGas
eth_feeHistory
eth_chainId
eth_getBlockByNumber
eth_getTransactionCount
eth_sendRawTransaction
eth_blockNumber
eth_getTransactionReceipt
eth_getBlockByNumber


DEBUG alloy_rpc_client.call 2024-11-08 06:02:25,379 call.rs:85 sending request method=eth_getTransactionReceipt id=11
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:25,379 reqwest_transport.rs:24 ReqwestTransport; url=http://localhost:3030/
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:02:25,380 pool.rs:270 reuse idle connection for ("http", localhost:3030)
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:02:25,381 pool.rs:396 pooling idle connection for ("http", localhost:3030)
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:25,381 reqwest_transport.rs:36 received response from server status=200 OK
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:25,381 reqwest_transport.rs:43 retrieved response body. Use `trace` for full body bytes=39
DEBUG alloy_rpc_client.call 2024-11-08 06:02:25,383 call.rs:85 sending request method=eth_blockNumber id=12
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:25,383 reqwest_transport.rs:24 Reqwe

eth_getTransactionReceipt
eth_blockNumber

    Transaction: 0x4d8e76ebd14622786969fec59e6277a03c01cc91ad9d3afa8f8a2735f60d91db
    Contract created: 0xba3c51fc0ff32e26fe3a6bc5f001d933bdee9dd3
    Gas used: 1232368

    Block Number: 272110989
    Block Hash: 0xe246c3e75b323f900f4baa09485813ae6f0c57a3301fae6c73872c3fe9fe1e2e
    Block Time: "Thu, 7 Nov 2024 22:02:22 +0000"



DEBUG alloy_rpc_client.call 2024-11-08 06:02:25,629 call.rs:85 sending request method=eth_getTransactionReceipt id=13
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:25,630 reqwest_transport.rs:24 ReqwestTransport; url=http://localhost:3030/
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:02:25,631 pool.rs:270 reuse idle connection for ("http", localhost:3030)
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:02:25,631 pool.rs:396 pooling idle connection for ("http", localhost:3030)
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:25,632 reqwest_transport.rs:36 received response from server status=200 OK
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:25,632 reqwest_transport.rs:43 retrieved response body. Use `trace` for full body bytes=1127
DEBUG ezkl.eth 2024-11-08 06:02:25,633 eth.rs:1146 scales: [
    13,
    13,
    13,
    13,
    13,
    13,
    13,
    13,
    13,
    13,
    13,
    13,
]
DEBUG ezkl.eth 2024-11-08 06:02:25,633 eth.rs

eth_getTransactionReceipt
eth_call
eth_call
eth_blockNumber
eth_call
eth_getBlockByNumber
eth_call
eth_call
eth_call
eth_call
eth_call
eth_call
eth_call
eth_call
eth_call
eth_call


Here we setup verifying and proving keys for the circuit. As the name suggests the proving key is needed for ... proving and the verifying key is needed for ... verifying. 

In [22]:
# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

DEBUG ezkl.pfsys.srs 2024-11-08 06:02:25,698 srs.rs:43 loading srs from "/Users/ethancemer/.ezkl/srs/kzg8.srs"
DEBUG ezkl.graph.vars 2024-11-08 06:02:25,699 vars.rs:425 number of blinding factors: 5
DEBUG ezkl.graph.vars 2024-11-08 06:02:25,699 vars.rs:452 model uses 3 advice blocks (size=1)
DEBUG ezkl.graph.vars 2024-11-08 06:02:25,699 vars.rs:460 model uses 1 fixed columns
DEBUG ezkl.graph.vars 2024-11-08 06:02:25,700 vars.rs:410 model uses [[1, 3, 2, 2], [3, 3, 3]] instance dims
DEBUG ezkl.graph.model 2024-11-08 06:02:25,700 model.rs:1023 configuring model
DEBUG ezkl.circuit.table 2024-11-08 06:02:25,700 table.rs:381 range check range: (-2, 2)
DEBUG ezkl.circuit.ops.chip 2024-11-08 06:02:25,700 chip.rs:920 assigning range check input
DEBUG ezkl.circuit.ops.chip 2024-11-08 06:02:25,700 chip.rs:925 assigning range check index
DEBUG ezkl.graph 2024-11-08 06:02:25,701 mod.rs:1582 degree: 3, log2_ceil of degrees: 2.0
DEBUG ezkl.graph 2024-11-08 06:02:25,701 mod.rs:1591 circuit size: 
 {


Now we generate a full proof. 

In [23]:
# GENERATE A PROOF

proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

DEBUG ezkl.graph 2024-11-08 06:02:25,767 mod.rs:900 rescaled and processed public inputs: {
  "inputs": [
    [
      "0x30644e72e131a029b85045b68181585d2833e84879b9709142df97797c38c001",
      "0x30644e72e131a029b85045b68181585d2833e84879b9709142df97791c378001",
      "0x30644e72e131a029b85045b68181585d2833e84879b9709142df9778bc364001",
      "0x30644e72e131a029b85045b68181585d2833e84879b9709142df97785c350001",
      "0x30644e72e131a029b85045b68181585d2833e84879b9709142df9777fc33c001",
      "0x30644e72e131a029b85045b68181585d2833e84879b9709142df97779c328001",
      "0x30644e72e131a029b85045b68181585d2833e84879b9709142df97773c314001",
      "0x30644e72e131a029b85045b68181585d2833e84879b9709142df9776dc300001",
      "0x30644e72e131a029b85045b68181585d2833e84879b9709142df97767c2ec001",
      "0x30644e72e131a029b85045b68181585d2833e84879b9709142df97761c2d8001",
      "0x30644e72e131a029b85045b68181585d2833e84879b9709142df9775bc2c4001",
      "0x30644e72e131a029b85045b68181585d2833e84879b

{'instances': [['01c0387c7997df429170b97948e833285d588181b64550b829a031e1724e6430', '0180371c7997df429170b97948e833285d588181b64550b829a031e1724e6430', '014036bc7897df429170b97948e833285d588181b64550b829a031e1724e6430', '0100355c7897df429170b97948e833285d588181b64550b829a031e1724e6430', '01c033fc7797df429170b97948e833285d588181b64550b829a031e1724e6430', '0180329c7797df429170b97948e833285d588181b64550b829a031e1724e6430', '0140313c7797df429170b97948e833285d588181b64550b829a031e1724e6430', '010030dc7697df429170b97948e833285d588181b64550b829a031e1724e6430', '01c02e7c7697df429170b97948e833285d588181b64550b829a031e1724e6430', '01802d1c7697df429170b97948e833285d588181b64550b829a031e1724e6430', '01402cbc7597df429170b97948e833285d588181b64550b829a031e1724e6430', '01002b5c7597df429170b97948e833285d588181b64550b829a031e1724e6430', '01300e530d5ea1439170b97948e833285d588181b64550b829a031e1724e6430', '01101c9e86c660439170b97948e833285d588181b64550b829a031e1724e6430', '01e00d3b0d5ea1439170b97948e8332

And verify it as a sanity check. 

In [24]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
    )

assert res == True
print("verified")

DEBUG ezkl.pfsys.srs 2024-11-08 06:02:25,828 srs.rs:33 loading srs from "/Users/ethancemer/.ezkl/srs/kzg8.srs"
DEBUG ezkl.pfsys 2024-11-08 06:02:25,828 mod.rs:806 loading verification key from "test.vk"
DEBUG ezkl.graph.vars 2024-11-08 06:02:25,829 vars.rs:425 number of blinding factors: 5
DEBUG ezkl.graph.vars 2024-11-08 06:02:25,829 vars.rs:452 model uses 3 advice blocks (size=1)
DEBUG ezkl.graph.vars 2024-11-08 06:02:25,829 vars.rs:460 model uses 1 fixed columns
DEBUG ezkl.graph.vars 2024-11-08 06:02:25,829 vars.rs:410 model uses [[1, 3, 2, 2], [3, 3, 3]] instance dims
DEBUG ezkl.graph.model 2024-11-08 06:02:25,830 model.rs:1023 configuring model
DEBUG ezkl.circuit.table 2024-11-08 06:02:25,830 table.rs:381 range check range: (-2, 2)
DEBUG ezkl.circuit.ops.chip 2024-11-08 06:02:25,830 chip.rs:920 assigning range check input
DEBUG ezkl.circuit.ops.chip 2024-11-08 06:02:25,830 chip.rs:925 assigning range check index
DEBUG ezkl.graph 2024-11-08 06:02:25,830 mod.rs:1582 degree: 3, log2_

verified


We can now create and then deploy a vanilla evm verifier.

In [25]:
abi_path = 'test.abi'
sol_code_path = 'test.sol'

res = await ezkl.create_evm_verifier(
        vk_path,
        settings_path,
        sol_code_path,
        abi_path,
    )
assert res == True

DEBUG ezkl.pfsys.srs 2024-11-08 06:02:25,838 srs.rs:33 loading srs from "/Users/ethancemer/.ezkl/srs/kzg8.srs"
DEBUG ezkl.pfsys 2024-11-08 06:02:25,839 mod.rs:806 loading verification key from "test.vk"
DEBUG ezkl.graph.vars 2024-11-08 06:02:25,839 vars.rs:425 number of blinding factors: 5
DEBUG ezkl.graph.vars 2024-11-08 06:02:25,840 vars.rs:452 model uses 3 advice blocks (size=1)
DEBUG ezkl.graph.vars 2024-11-08 06:02:25,840 vars.rs:460 model uses 1 fixed columns
DEBUG ezkl.graph.vars 2024-11-08 06:02:25,841 vars.rs:410 model uses [[1, 3, 2, 2], [3, 3, 3]] instance dims
DEBUG ezkl.graph.model 2024-11-08 06:02:25,841 model.rs:1023 configuring model
DEBUG ezkl.circuit.table 2024-11-08 06:02:25,842 table.rs:381 range check range: (-2, 2)
DEBUG ezkl.circuit.ops.chip 2024-11-08 06:02:25,843 chip.rs:920 assigning range check input
DEBUG ezkl.circuit.ops.chip 2024-11-08 06:02:25,843 chip.rs:925 assigning range check index
DEBUG ezkl.graph 2024-11-08 06:02:25,844 mod.rs:1582 degree: 3, log2_

In [26]:
import json

addr_path_verifier = "addr_verifier.txt"

res = await ezkl.deploy_evm(
    addr_path_verifier,
    sol_code_path,
    'http://127.0.0.1:3030'
)

assert res == True

DEBUG alloy_rpc_client.call 2024-11-08 06:02:25,943 call.rs:85 sending request method=eth_chainId id=0
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:25,944 reqwest_transport.rs:24 ReqwestTransport; url=http://127.0.0.1:3030/
DEBUG reqwest.connect 2024-11-08 06:02:25,944 connect.rs:497 starting new connection: http://127.0.0.1:3030/
DEBUG hyper_util.client.legacy.connect.http 2024-11-08 06:02:25,945 http.rs:643 connecting to 127.0.0.1:3030
DEBUG hyper_util.client.legacy.connect.http 2024-11-08 06:02:25,946 http.rs:646 connected to 127.0.0.1:3030
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:02:25,946 pool.rs:396 pooling idle connection for ("http", 127.0.0.1:3030)
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:25,947 reqwest_transport.rs:36 received response from server status=200 OK
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:25,948 reqwest_transport.rs:43 retrieved response body. Use `trace` for full body bytes=42
INFO ezkl.eth 2024

eth_chainId


DEBUG foundry_compilers.compile 2024-11-08 06:02:26,034 mod.rs:458 finished output.status=exit status: 0 output.stderr=""
DEBUG ezkl.eth 2024-11-08 06:02:26,035 eth.rs:1188 runtime bytecode size: 9744
DEBUG alloy_rpc_client.call 2024-11-08 06:02:26,036 call.rs:85 sending request method=eth_estimateGas id=1
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:26,036 reqwest_transport.rs:24 ReqwestTransport; url=http://127.0.0.1:3030/
DEBUG alloy_rpc_client.call 2024-11-08 06:02:26,037 call.rs:85 sending request method=eth_feeHistory id=2
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:26,037 reqwest_transport.rs:24 ReqwestTransport; url=http://127.0.0.1:3030/
DEBUG alloy_rpc_client.call 2024-11-08 06:02:26,037 call.rs:85 sending request method=eth_chainId id=3
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:26,037 reqwest_transport.rs:24 ReqwestTransport; url=http://127.0.0.1:3030/
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:02:26,038 pool.rs:2

eth_estimateGas
eth_chainId
eth_feeHistory
eth_getBlockByNumber
eth_getTransactionCount
eth_sendRawTransaction
eth_blockNumber
eth_getTransactionReceipt
eth_getBlockByNumber


DEBUG alloy_rpc_client.call 2024-11-08 06:02:26,304 call.rs:85 sending request method=eth_getTransactionReceipt id=10
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:26,305 reqwest_transport.rs:24 ReqwestTransport; url=http://127.0.0.1:3030/
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:02:26,305 pool.rs:270 reuse idle connection for ("http", 127.0.0.1:3030)
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:02:26,306 pool.rs:396 pooling idle connection for ("http", 127.0.0.1:3030)
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:26,307 reqwest_transport.rs:36 received response from server status=200 OK
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:26,307 reqwest_transport.rs:43 retrieved response body. Use `trace` for full body bytes=39
DEBUG alloy_rpc_client.call 2024-11-08 06:02:26,308 call.rs:85 sending request method=eth_blockNumber id=11
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:26,308 reqwest_transport.rs:24 Reqwe

eth_getTransactionReceipt
eth_blockNumber

    Transaction: 0xad57cf85a7c6721ce456b17231fcf3e05d11ec6976e5322bed80701adf7021c4
    Contract created: 0x9bda88da960e08cc166d3e824109b5af3e376278
    Gas used: 2159820

    Block Number: 272110990
    Block Hash: 0xd9ea25a3a899f022045e8b06bb7542081dd539d2b6781bf1ed09b980140c4acb
    Block Time: "Thu, 7 Nov 2024 22:02:23 +0000"



DEBUG alloy_rpc_client.call 2024-11-08 06:02:26,554 call.rs:85 sending request method=eth_getTransactionReceipt id=12
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:26,555 reqwest_transport.rs:24 ReqwestTransport; url=http://127.0.0.1:3030/
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:02:26,556 pool.rs:270 reuse idle connection for ("http", 127.0.0.1:3030)
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:02:26,556 pool.rs:396 pooling idle connection for ("http", 127.0.0.1:3030)
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:26,557 reqwest_transport.rs:36 received response from server status=200 OK
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:02:26,557 reqwest_transport.rs:43 retrieved response body. Use `trace` for full body bytes=1127
INFO ezkl.execute 2024-11-08 06:02:26,557 execute.rs:1620 Contract deployed at: 0x9bda88da960e08cc166d3e824109b5af3e376278


eth_getTransactionReceipt


With the vanilla verifier deployed, we can now create the data attestation contract, which will read in the instances from the calldata to the verifier, attest to them, call the verifier and then return the result. 

In [27]:

abi_path = 'test.abi'
sol_code_path = 'test.sol'
input_path = 'input.json'

res = await ezkl.create_evm_data_attestation(
        input_path,
        settings_path,
        sol_code_path,
        abi_path,
    )

DEBUG alloy_rpc_client.poller 2024-11-08 06:02:26,561 poller.rs:158 client dropped
DEBUG tracing.span 2024-11-08 06:02:26,562 mod.rs:1396 read;
DEBUG tracing.span 2024-11-08 06:02:26,563 mod.rs:470 version;
DEBUG foundry_compilers.compile 2024-11-08 06:02:26,563 mod.rs:475 getting Solc version cmd="/Users/ethancemer/Library/Application Support/svm/0.8.20/solc-0.8.20" "--version"
DEBUG foundry_compilers.compile 2024-11-08 06:02:26,567 mod.rs:479 version=0.8.20+commit.a1b79de6.Darwin.appleclang
DEBUG tracing.span 2024-11-08 06:02:26,567 mod.rs:443 compile;
DEBUG foundry_compilers.compile 2024-11-08 06:02:26,568 mod.rs:448 compiling cmd="/Users/ethancemer/Library/Application Support/svm/0.8.20/solc-0.8.20" "--standard-json"
DEBUG foundry_compilers.compile 2024-11-08 06:02:26,568 mod.rs:451 spawned
DEBUG foundry_compilers.compile 2024-11-08 06:02:26,569 mod.rs:455 wrote JSON input to stdin
DEBUG foundry_compilers.compile 2024-11-08 06:02:27,334 mod.rs:458 finished output.status=exit status

Now we can deploy the data attest verifier contract. For security reasons, this binding will only deploy to a local anvil instance, using accounts generated by anvil. 
So should only be used for testing purposes.

In [30]:
addr_path_da = "addr_da.txt"

res = await ezkl.deploy_da_evm(
        addr_path_da,
        input_path,
        settings_path,
        sol_code_path,
        RPC_URL,
    )


DEBUG alloy_rpc_client.call 2024-11-08 06:03:42,175 call.rs:85 sending request method=eth_chainId id=0
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:03:42,176 reqwest_transport.rs:24 ReqwestTransport; url=http://localhost:3030/
DEBUG reqwest.connect 2024-11-08 06:03:42,176 connect.rs:497 starting new connection: http://localhost:3030/
DEBUG hyper_util.client.legacy.connect.dns 2024-11-08 06:03:42,177 dns.rs:122 resolving host="localhost"
DEBUG hyper_util.client.legacy.connect.http 2024-11-08 06:03:42,178 http.rs:643 connecting to 127.0.0.1:3030
DEBUG hyper_util.client.legacy.connect.http 2024-11-08 06:03:42,179 http.rs:646 connected to 127.0.0.1:3030
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:03:42,179 pool.rs:396 pooling idle connection for ("http", localhost:3030)
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:03:42,180 reqwest_transport.rs:36 received response from server status=200 OK
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:03:42,1

eth_chainId


DEBUG foundry_compilers.compile 2024-11-08 06:03:42,931 mod.rs:458 finished output.status=exit status: 0 output.stderr=""
DEBUG ezkl.eth 2024-11-08 06:03:42,937 eth.rs:1188 runtime bytecode size: 4525
DEBUG ezkl.eth 2024-11-08 06:03:42,938 eth.rs:648 call_data: 0x1f3be514000000000000000000000000c6962004f452be9203591991d15f6b388e09e8d00000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000000b000000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000000900000000000000000000000000000000000000000000000000000000000000080000000000000000000000000000000000000000000000000000000000000007000000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000050000000000000000000000000000000000000000000000000000000000000004000000000000000000000000

eth_estimateGas
eth_feeHistory
eth_chainId
eth_getBlockByNumber
eth_getTransactionCount
eth_sendRawTransaction
eth_blockNumber
eth_getTransactionReceipt
eth_getBlockByNumber


DEBUG alloy_rpc_client.call 2024-11-08 06:03:43,211 call.rs:85 sending request method=eth_getTransactionReceipt id=10
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:03:43,212 reqwest_transport.rs:24 ReqwestTransport; url=http://localhost:3030/
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:03:43,212 pool.rs:270 reuse idle connection for ("http", localhost:3030)
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:03:43,213 pool.rs:396 pooling idle connection for ("http", localhost:3030)
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:03:43,213 reqwest_transport.rs:36 received response from server status=200 OK
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:03:43,213 reqwest_transport.rs:43 retrieved response body. Use `trace` for full body bytes=39
DEBUG alloy_rpc_client.call 2024-11-08 06:03:43,214 call.rs:85 sending request method=eth_blockNumber id=11
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:03:43,214 reqwest_transport.rs:24 Reqwe

eth_getTransactionReceipt
eth_blockNumber

    Transaction: 0x671171ad4cdaf0bf69508577ad32e956910d4d8dbdb68e8600f49512e31632ee
    Contract created: 0x0e03350d11f24839373262143b46d8d68e5e6649
    Gas used: 1785294

    Block Number: 272110994
    Block Hash: 0x56d9b1f6e38d18073d6d51394eed21f162405b769081de31dd79bb0233cf2c6e
    Block Time: "Thu, 7 Nov 2024 22:03:39 +0000"



DEBUG alloy_rpc_client.call 2024-11-08 06:03:43,460 call.rs:85 sending request method=eth_getTransactionReceipt id=12
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:03:43,461 reqwest_transport.rs:24 ReqwestTransport; url=http://localhost:3030/
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:03:43,461 pool.rs:270 reuse idle connection for ("http", localhost:3030)
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:03:43,462 pool.rs:396 pooling idle connection for ("http", localhost:3030)
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:03:43,462 reqwest_transport.rs:36 received response from server status=200 OK
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:03:43,463 reqwest_transport.rs:43 retrieved response body. Use `trace` for full body bytes=1127
INFO ezkl.execute 2024-11-08 06:03:43,463 execute.rs:1590 Contract deployed at: 0x0e03350D11F24839373262143B46D8d68E5E6649


eth_getTransactionReceipt


DEBUG alloy_rpc_client.poller 2024-11-08 06:03:43,467 poller.rs:158 client dropped


Call the view only verify method on the contract to verify the proof. Since it is a view function this is safe to use in production since you don't have to pass your private key.

In [32]:
!export RUST_BACKTRACE=1

calls_to_account = on_chain_data(x)

data = dict(input_data =  {'rpc': RPC_URL, 'calls': calls_to_account })

# Serialize on-chain data into file:
json.dump(data, open("input.json", 'w'))

witness_path = "witness.json"

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
    )

print(res)
assert os.path.isfile(proof_path)
# read the verifier address
addr_verifier = None
with open(addr_path_verifier, 'r') as f:
    addr = f.read()
#read the data attestation address
addr_da = None
with open(addr_path_da, 'r') as f:
    addr_da = f.read()

res = await ezkl.verify_evm(
    addr,
    proof_path,
    RPC_URL,
    addr_da,
)

DEBUG web3.manager.RequestManager 2024-11-08 06:04:51,861 manager.py:331 Making request. Method: eth_sendTransaction
DEBUG web3.manager.RequestManager 2024-11-08 06:04:51,861 manager.py:331 Making request. Method: eth_getBlockByNumber
DEBUG web3.providers.HTTPProvider 2024-11-08 06:04:51,862 rpc.py:166 Making request HTTP. URI: http://localhost:3030, Method: eth_getBlockByNumber
DEBUG urllib3.connectionpool 2024-11-08 06:04:51,864 connectionpool.py:546 http://localhost:3030 "POST / HTTP/11" 200 1564
DEBUG web3.providers.HTTPProvider 2024-11-08 06:04:51,864 rpc.py:172 Getting response HTTP. URI: http://localhost:3030, Method: eth_getBlockByNumber, Response: {'jsonrpc': '2.0', 'id': 66, 'result': {'hash': '0xd80ef581b901ceb015834caf7f77867b45351fd69577c062962aa5a3a878c15c', 'parentHash': '0x61723b9baea12832aaa30b9861cabca44be6db350ebc254ac067c6d2529e43ce', 'sha3Uncles': '0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347', 'miner': '0x0000000000000000000000000000000000000

eth_getBlockByNumber
eth_chainId
eth_chainId
eth_estimateGas
eth_chainId
eth_getBlockByNumber
eth_sendTransaction
eth_accounts
eth_chainId
eth_getTransactionReceipt


DEBUG web3.manager.RequestManager 2024-11-08 06:04:51,987 manager.py:331 Making request. Method: eth_getTransactionReceipt
DEBUG web3.providers.HTTPProvider 2024-11-08 06:04:51,987 rpc.py:166 Making request HTTP. URI: http://localhost:3030, Method: eth_getTransactionReceipt
DEBUG urllib3.connectionpool 2024-11-08 06:04:51,989 connectionpool.py:546 http://localhost:3030 "POST / HTTP/11" 200 39
DEBUG web3.providers.HTTPProvider 2024-11-08 06:04:51,989 rpc.py:172 Getting response HTTP. URI: http://localhost:3030, Method: eth_getTransactionReceipt, Response: {'jsonrpc': '2.0', 'id': 75, 'result': None}


eth_getTransactionReceipt


DEBUG web3.manager.RequestManager 2024-11-08 06:04:52,090 manager.py:331 Making request. Method: eth_getTransactionReceipt
DEBUG web3.providers.HTTPProvider 2024-11-08 06:04:52,090 rpc.py:166 Making request HTTP. URI: http://localhost:3030, Method: eth_getTransactionReceipt
DEBUG urllib3.connectionpool 2024-11-08 06:04:52,092 connectionpool.py:546 http://localhost:3030 "POST / HTTP/11" 200 39
DEBUG web3.providers.HTTPProvider 2024-11-08 06:04:52,092 rpc.py:172 Getting response HTTP. URI: http://localhost:3030, Method: eth_getTransactionReceipt, Response: {'jsonrpc': '2.0', 'id': 76, 'result': None}


...

eth_getTransactionReceipt
eth_blockNumber

    Transaction: 0x31a6594520b80d51048e5ce28ea59479492e799d79b986889d53f09f5c7b79f4
    Contract created: 0x9674f70c5ceb61f990977d325abf2c0201a4c520
    Gas used: 1232368

    Block Number: 272110998
    Block Hash: 0xb64811a5c7b764e6429aae750cd74f41fc2d6412673ae58c1a044b597155fa0d
    Block Time: "Thu, 7 Nov 2024 22:04:49 +0000"



DEBUG alloy_rpc_client.call 2024-11-08 06:04:52,874 call.rs:85 sending request method=eth_getTransactionReceipt id=13
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:04:52,875 reqwest_transport.rs:24 ReqwestTransport; url=http://localhost:3030/
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:04:52,875 pool.rs:270 reuse idle connection for ("http", localhost:3030)
DEBUG hyper_util.client.legacy.pool 2024-11-08 06:04:52,876 pool.rs:396 pooling idle connection for ("http", localhost:3030)
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:04:52,876 reqwest_transport.rs:36 received response from server status=200 OK
DEBUG alloy_transport_http.reqwest_transport 2024-11-08 06:04:52,877 reqwest_transport.rs:43 retrieved response body. Use `trace` for full body bytes=1127
DEBUG ezkl.eth 2024-11-08 06:04:52,877 eth.rs:1146 scales: [
    13,
    13,
    13,
    13,
    13,
    13,
    13,
    13,
    13,
    13,
    13,
    13,
]
DEBUG ezkl.eth 2024-11-08 06:04:52,877 eth.rs

eth_getTransactionReceipt
eth_call
eth_call
eth_blockNumber
eth_call
eth_getBlockByNumber
eth_call
eth_call
eth_call
eth_call
eth_call
eth_call
eth_call
eth_call
eth_call
eth_call
{'instances': [['01807e9b4197df429170b97948e833285d588181b64550b829a031e1724e6430', '01407d3b4197df429170b97948e833285d588181b64550b829a031e1724e6430', '01007cdb4097df429170b97948e833285d588181b64550b829a031e1724e6430', '01c07a7b4097df429170b97948e833285d588181b64550b829a031e1724e6430', '0180791b4097df429170b97948e833285d588181b64550b829a031e1724e6430', '014078bb3f97df429170b97948e833285d588181b64550b829a031e1724e6430', '0100775b3f97df429170b97948e833285d588181b64550b829a031e1724e6430', '01c075fb3e97df429170b97948e833285d588181b64550b829a031e1724e6430', '0180749b3e97df429170b97948e833285d588181b64550b829a031e1724e6430', '0140733b3e97df429170b97948e833285d588181b64550b829a031e1724e6430', '010072db3d97df429170b97948e833285d588181b64550b829a031e1724e6430', '01c0707b3d97df429170b97948e833285d588181b64550b829a031e

RuntimeError: Failed to run verify_evm: [eth] a transport error occurred: server returned an error response: error code 3: execution reverted: revert: Public input does not match

DEBUG alloy_rpc_client.poller 2024-11-08 06:04:53,139 poller.rs:158 client dropped
